In [1]:
import xmlrpc.client
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

load_dotenv()

dbUrl = os.environ["ODOO_URL"]
dbName = os.environ["ODOO_NAME"]
dbUsername = os.environ["ODOO_USERNAME"]
dbPassword = os.environ["ODOO_PASSWORD"]

common = xmlrpc.client.ServerProxy('{}/xmlrpc/2/common'.format(dbUrl))
uid = common.authenticate(dbName, dbUsername, dbPassword, {})
models = xmlrpc.client.ServerProxy('{}/xmlrpc/2/object'.format(dbUrl))

# Data Handling Functions
def id_field_into_1value(jsonFile):
    for index, line in enumerate(jsonFile):
        for item in line:
            if type(line[item]) == list:
                jsonFile[index][item] = jsonFile[index][item][0]

df = models.execute_kw(
    dbName,
    uid,
    dbPassword,
    'res.company',
    'search_read',
    [],
    {'fields': ["name", "account_fiscal_country_id" ] })

id_field_into_1value(df)

df = pd.DataFrame(df)

df = df.rename(columns={
    'id': 'company_id',
    'name': 'company_name'
})

df2 = models.execute_kw(
    dbName,
    uid,
    dbPassword,
    'res.country',
    'search_read',
    [],
    {'fields': ["name"] })

id_field_into_1value(df2)

df2 = pd.DataFrame(df2)

df2 = df2.rename(columns={
    'id': 'country_id',
    'name': 'country_name'
})

df3 = df.merge(df2, how='left', left_on='account_fiscal_country_id', right_on='country_id')

df4 = models.execute_kw(
    dbName,
    uid,
    dbPassword,
    'res.partner',
    'search_read',
    [],
    {'fields': [
        "__last_update",
        "name",
        "display_name",
        #"company_name",
        #"title",
        "commercial_company_name",
        "commercial_partner_id",
        "x_studio_tipo",
        "x_studio_tipo_de_cliente",
        "x_studio_razn_social",
        "company_id",
        "contact_address",
        "street",
        "street2",
        "city",
        "zip",
        "x_studio_regin_comercial",
        "x_hubspot_id",
        "currency_id",
        "email",
        #"industry_id",
        #"l10n_co_edi_commercial_name",
        "sale_order_count",
        "subscription_count"
    ]}
)

id_field_into_1value(df4)

df4 = pd.DataFrame(df4)

df4 = df4.rename(columns={
    'id': 'contact_id',
    'name': 'contact_name',
    "x_studio_tipo": 'contact_type',
    "x_studio_tipo_de_cliente": 'tipo_de_cliente',
    "x_studio_razn_social": 'razon_social',
    "x_studio_regin_comercial": 'region_comercial'
})

df5 = df3.merge(df4, how='left', left_on='company_id', right_on='company_id')

DATABASE_CONNECTION_URI = os.environ["DB_URL"]
# create a connection to the database
engine = create_engine(DATABASE_CONNECTION_URI)

# insert the dataframe into the postgres table
df5.to_sql('odoo_contacts', con=engine, if_exists='replace', index=False)

960